# Gestion des biblothèques

In [1]:
import pandas as pd
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Pré traitement des données

#### Importation du dataset

In [2]:
recipes_df = pd.read_csv('../../data/raw/RAW_recipes.csv', on_bad_lines='skip', engine="python")

### Suppression des colonnes inutiles

In [3]:
recipes_df = recipes_df.drop(['submitted', 'contributor_id', 'minutes', 'nutrition', 'n_steps', 'n_ingredients', 'steps'], axis=1)
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         231636 non-null  object
 1   id           231637 non-null  int64 
 2   tags         231637 non-null  object
 3   description  226658 non-null  object
 4   ingredients  231637 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.8+ MB


#### Suppression des lignes incorrectes

In [4]:
recipes_df = recipes_df.drop_duplicates()
recipes_df = recipes_df.dropna()

#### Sélection des 20000 premières recettes

In [5]:
recipes_df = recipes_df.head(20000)

#### Formattage du titre

Les titres sont rentrés manuellement par les utilisateurs. Ils contiennent parfois des espaces en trop, que nous allons chercher à supprimer

In [6]:
# Retourner le titre avec un seul espace entre les mots.
def clean_recipe_titles(title):
  words = title.split()
  cleaned_title = ' '.join(words)
  return cleaned_title

In [7]:
recipes_df.name = recipes_df.name.apply(clean_recipe_titles)

#### Nettoyage des ingrédients

In [8]:
def text_cleaning(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

recipes_df['ingredients'] = recipes_df['ingredients'].apply(lambda x: text_cleaning(x))

#### Création d'une colonne (tags + ingrédients + description)

Au lieu de faire uniquement la recherche de similarité sur la colonne des ingrédients, nous allons ajouter le contexte avec la description de la recette et les différents tags (healthy, fast, etc.)

In [9]:
def assemble_categories(row):
    return row['tags'] + " " + row['ingredients'] + " " + row['description']

In [10]:
recipes_df['combined_features'] = recipes_df.apply(assemble_categories, axis=1)

#### Vérification du tri

In [11]:
recipes_df.duplicated().sum()
recipes_df.isnull().sum()

,0
name,0
id,0
tags,0
description,0
ingredients,0
combined_features,0


#### Affichage des la dimension du dataframe

In [12]:
(20000, 3)

(20000, 3)

# Filtrage par contenu

### Utilisation de TF-IDF

On accepte 20000 mots de vocabulaire au maximum et on supprime les particules anglaises

In [13]:
tfidf = TfidfVectorizer(stop_words='english', max_features=20000)

On applique le procession à cette nouvelle colonne

In [14]:
tfidf_matrix = tfidf.fit_transform(recipes_df['combined_features'])
tfidf_matrix.shape

(20000, 20000)

### Création du linear kernel

On calcule les angles entre la matrice TF-IDF en ligne et en colonne

In [15]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Création des indices

Les indices permettes de rechercher dans le dataframe des recettes. Les id ne correspondant plus à l'index du tableau

In [16]:
indices = pd.Series(recipes_df.index, index=recipes_df['name']).drop_duplicates()
indices

,0
name,
arriba baked winter squash mexican style,0
a bit different breakfast pizza,1
all in the kitchen chili,2
alouette potatoes,3
amish tomato ketchup for canning,4
...,...
beef parmesan,20493
beef pasanda,20494
beef pasta bake,20495


# Le programme principale

In [17]:
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
      # On récupère la ligne des scores correspondant à la recette
      sim_scores = list(enumerate(cosine_sim[indices[title]]))

      # On trie cette liste du plus grand score au plus faible
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

      # On retient que les 5 premiers scores (on a gardé la recette originelle pour comparer)
      sim_scores = sim_scores[0:6]

      # Transformation de la liste en dataframe
      food_indices = [i[0] for i in sim_scores]

      df_reco = pd.DataFrame({'index': food_indices, 'similarity': [i[1] for i in sim_scores]})
      df_reco = pd.merge(df_reco, recipes_df[['name', 'ingredients', 'description', 'tags']], left_on='index', right_index=True)
      df_reco = df_reco[['name', 'ingredients', 'description', 'tags', 'similarity']]

      return df_reco

    except:
      print("La recette n'existe pas dans la base de données")

### Lancement du programme

In [19]:
recette = input("Entrez le nom d'une recette : (ex. fool the meat eaters chili)")
print_df = get_recommendations(recette)

if print_df is not None:
   display(print_df.head())

Entrez le nom d'une recette : (ex. fool the meat eaters chili)fool the meat eaters chili m
La recette n'existe pas dans la base de données
